## Get data from URL and save

In [1]:
# download dataset
def get_data(link):
    
    from zipfile import ZipFile
    import requests
    import os
    
    output = 'temp.zip'

    response = requests.get(link, stream=True)
    handle = open(output, "wb")
    for chunk in response.iter_content(chunk_size=512):
        if chunk:
            handle.write(chunk)
    handle.close()


    with ZipFile(output,"r") as zip_file:
        for name in zip_file.namelist():
            if name != '_about.txt':
                filename = name
        zip_file.extractall("data")
    
    os.remove('temp.zip')
    os.remove('data/_about.txt')
    return filename

In [2]:
path = 'data/' + get_data('http://www.manythings.org/anki/spa-eng.zip')

In [3]:
path

'data/spa.txt'

## Clean data

In [10]:
def clean_lines(line):
    
    from unicodedata import normalize
    import string
    
    remove_punct_map = dict.fromkeys(map(ord, string.punctuation)) # thank you Reed!
    
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [word.lower() for word in line]
    line = [word.translate(remove_punct_map) for word in line]
    line = [word for word in line if word.isalpha()]
    
    return(line)

In [11]:
def clean_data(filepath):
    
    import numpy as np
    
    import re
    
    file = open(filepath, mode='rt')
    text = file.read()
    file.close()
    
    lines = text.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    
    all_cleaned = []
    

    
    for pair in pairs:
        cleaned_pair = []
        for sentence in pair:
            clean_sentence = clean_lines(sentence)
            cleaned_pair.append(' '.join(clean_sentence))
            
        all_cleaned.append(cleaned_pair)
    
    no_duplicates = []
    previous_line = ''
    
    for line in all_cleaned:
        if (line[0] != previous_line) and (' tom' not in line[0]) and ('tom ' not in line[0]):
            no_duplicates.append(line)
            previous_line = line[0] 
          
        
    return np.array(no_duplicates)

In [12]:
clean_data = clean_data(path)

In [13]:
clean_data[1600:1610]

array([['i like chess', 'me gusta el ajedrez'],
       ['i like fruit', 'me gusta la fruta'],
       ['i like honey', 'me gusta la miel'],
       ['i like opera', 'me gusta la opera'],
       ['i like pizza', 'me gusta la pizza'],
       ['i like sushi', 'me gusta el sushi'],
       ['i like these', 'me gusta eso'],
       ['i like women', 'me gustan las mujeres'],
       ['i live alone', 'yo vivo solo'],
       ['i love books', 'me encantan los libros']], dtype='<U275')

In [14]:
clean_data.shape

(83783, 2)

## Split in test and train

In [18]:
def split_data(dataset):
    import numpy as np
    from sklearn.externals import joblib
    
    lines_number = 10000
    new_set = dataset[:lines_number, :]
    
    np.random.shuffle(new_set)
    train, test = new_set[:9000], new_set[9000:]
    
    

    joblib.dump(new_set, 'data/dataset.pkl')
    joblib.dump(train, 'data/train.pkl')
    joblib.dump(test, 'data/test.pkl')
    
    return new_set, train, test

In [19]:
dataset, train, test = split_data(clean_data)

In [45]:
len(train)

9000

## Tokenize data

In [27]:
from keras.preprocessing.text import Tokenizer

/Users/sonik/anaconda3/envs/pythondata/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [48]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(dataset[:,0])
eng_length = max(len(line.split()) for line in dataset[:,0])
eng_vocab_size = len(eng_tokenizer.word_counts) + 1
targ_tokenizer = Tokenizer()
targ_tokenizer.fit_on_texts(dataset[:,1])
targ_length = max(len(line.split()) for line in dataset[:,1])
targ_vocab_size = len(targ_tokenizer.word_counts) + 1
print(f'a number of unique words in english dataset - {len(eng_tokenizer.word_counts)}')
print(f'max sentence size english dataset - {eng_length}')
print(f'a number of unique words in target dataset - {len(targ_tokenizer.word_counts)}')

a number of unique words in english dataset - 2957
vocabulary size english dataset - 2958
max sentence size english dataset - 5
a number of unique words in target dataset - 4649


In [47]:
max(len(line.split()) for line in dataset[:,1])

9

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

In [ ]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y
 
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model


In [ ]:
dataset = load_clean_sentences('english-spanish-both.pkl')
train = load_clean_sentences('english-spanish-train.pkl')
shuffle(train)
test = load_clean_sentences('english-spanish-test.pkl')
shuffle(test)

In [ ]:
test

In [ ]:
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
spa_tokenizer = create_tokenizer(dataset[:, 1])
spa_vocab_size = len(spa_tokenizer.word_index) + 1
spa_length = max_length(dataset[:, 1])
print('Spanish Vocabulary Size: %d' % spa_vocab_size)
print('Spanish Max Length: %d' % (spa_length))

In [ ]:
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_sequences(spa_tokenizer, spa_length, train[:, 1])
trainY = encode_output(trainY, spa_vocab_size)
# prepare validation data
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_sequences(spa_tokenizer, spa_length, test[:, 1])
testY = encode_output(testY, spa_vocab_size)

In [ ]:
model = define_model(eng_vocab_size, spa_vocab_size, eng_length, spa_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
filename = 'model1.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)
